<b>Inspiration</b>

The dataset is large and informative, I believe you can have a lot of fun with it! Let me put some ideas below to futher inspire kagglers!

Fit a regression model on reviews and score to see which words are more indicative to a higher/lower score
Perform a sentiment analysis on the reviews
Find correlation between reviewer's nationality and scores.
Beautiful and informative visualization on the dataset.
Clustering hotels based on reviews
Simple recommendation engine to the guest who is fond of a special characteristic of hotel.

In [1]:
# Importing Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

# NLP Packages
import nltk 
from textblob import TextBlob 
from textblob import Word
import re
import string

# WordCloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# Sklearn Packages
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text 
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, precision_score, f1_score, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression

# ImbLearn Packages
from imblearn.over_sampling import SMOTE

# Pandas Settings
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows', 100)

# Solve warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# Import csv file
df = pd.read_csv('csv/Hotel_Reviews.csv')
df_US = pd.read_csv('csv/Hotel_Reviews_US.csv')

# Data Cleaning and EDA

## Understand Dataset

In [3]:
# Taking a lot at the dataset
df.head(5)

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [4]:
# Checking the shape of the dataframe
df.shape

(515738, 17)

In [5]:
# df = df.sample(frac=0.05)
# df.shape

In [6]:
# Checking null values
df.isna().sum()

Hotel_Address                                    0
Additional_Number_of_Scoring                     0
Review_Date                                      0
Average_Score                                    0
Hotel_Name                                       0
Reviewer_Nationality                             0
Negative_Review                                  0
Review_Total_Negative_Word_Counts                0
Total_Number_of_Reviews                          0
Positive_Review                                  0
Review_Total_Positive_Word_Counts                0
Total_Number_of_Reviews_Reviewer_Has_Given       0
Reviewer_Score                                   0
Tags                                             0
days_since_review                                0
lat                                           3268
lng                                           3268
dtype: int64

In [7]:
# Checking how many hotels in this dataset
len(df.Hotel_Name.unique())

1492

In [8]:
# Checking the hotel with the highest number of reviews
df.pivot_table(index=['Hotel_Name'], aggfunc='size').nlargest()

Hotel_Name
Britannia International Hotel Canary Wharf           4789
Strand Palace Hotel                                  4256
Park Plaza Westminster Bridge London                 4169
Copthorne Tara Hotel London Kensington               3578
DoubleTree by Hilton Hotel London Tower of London    3212
dtype: int64

In [9]:
# Double checking if the number matches to the column Total_Number_of_Reviews
df[df['Hotel_Name'] == 'Britannia International Hotel Canary Wharf'].head(2)

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
63942,163 Marsh Wall Docklands Tower Hamlets London ...,2682,8/3/2017,7.1,Britannia International Hotel Canary Wharf,United Kingdom,The car park was small and unpleasant People ...,31,9086,The location was excellent for getting to the O2,10,3,7.9,"[' Leisure trip ', ' Group ', ' Standard Doubl...",0 days,51.50191,-0.023221
63943,163 Marsh Wall Docklands Tower Hamlets London ...,2682,8/3/2017,7.1,Britannia International Hotel Canary Wharf,United Kingdom,We weren t told that the only spa facility op...,34,9086,The house keeping lady made my boyfriends day...,14,3,8.3,"[' Leisure trip ', ' Couple ', ' Standard Doub...",0 days,51.50191,-0.023221


### Findings:

- There are reviews from 1,492 hotels
- The data is fairly clean. It doesn't much null values
- It's missing the cities where the hotels are located.
- There are reviews without the latitude and longitude.
- The actual number of reviews per hotel does not match to the actual number

## Data Cleaning

In [10]:
# Checking rows where the values are null
len(df[['Hotel_Address']][df.isnull().any(axis=1)])

3268

### Remove Punctuation

In [11]:
df.columns

Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
       'days_since_review', 'lat', 'lng'],
      dtype='object')

In [12]:
# This function lowercase all the review words, removes punctuation and numbers
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)

    return text

round1 = lambda x: clean_text_round1(x)

In [13]:
# Applying clean_text_round1 function
df['Negative_Review_Clean'] = pd.DataFrame(df.Negative_Review.apply(round1))
df['Positive_Review_Clean'] = pd.DataFrame(df.Negative_Review.apply(round1))

In [14]:
# Create a new column only with English words
words = set(nltk.corpus.words.words())
df['Negative_Review_Clean2'] = ''

for index in df.index:
    sent = df.Negative_Review_Clean[index]
    df['Negative_Review_Clean2'][index] = " ".join(w for w in nltk.wordpunct_tokenize(sent) if w.lower() in words or not w.isalpha())
    
    # 32 minutes

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


#### Findings and Takeaways:

- There are 17 hotels without latitude and longitude. I'll work on it as a stretch goal

## NEED TO WORK ON THAT - Fix Spelling

- To do a spell check, I will choose a random review and check if there is any misspells in it
- I will create a function that will use TextBlob to fix misspellings
- Check the result

It seems that there are a few misspellings, such as the words `theough` and `extreamly`. I'll use TextBlob to fix them.

In [15]:
# Create function to fix misspells
# Create a function to get subjectivity
def spellcheck(text):
    return Word(text).spellcheck

# def spellcheck(text):
    

In [16]:
# # Checking if function works
# spellcheck(df['Negative_Review'][436517])

In [17]:
# w = Word(df['Negative_Review'][4])
# w.spellcheck()

In [18]:
# df['Negative_Review_SC'] = df['Negative_Review'].apply(spellcheck)

In [19]:
# df.Negative_Review[4]

In [20]:
# blob = TextBlob(df.Negative_Review[4])
# blob.correct()

### Findings and Takeaways:
- While checking a random 

# Data Engineering

## Create a function for Sentiment Analysis

In this step, I will generate a sentiment analysis. Normally, this would be a step that I'd run after data cleaning for NLP. However, previous tests showed me that data cleaning does not affect the sentiment analysis using TextBlob.

Running sentiment analysis takes a lot of time because I have more than 515K observations. For this reason, once the sentiment analysis is created, I will pickle the DataFrame and upload it again, so it won't run again.

In [21]:
# Create a function to get subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# Create a function to get polarity with tweets
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

<b>NOTE:</b>

Each of the two following cells takes around 10 minutes to run. For this reason, I will sabe the DataFrame into a csv file and upload it again.

In [22]:
# Create new columns to compare polarity and subjetivity on Negative Reviews
df['Polarity_Net'] = df['Negative_Review'].apply(getPolarity)
df['Polarity_Pos'] = df['Positive_Review'].apply(getPolarity)

# 8 minutes

In [23]:
# Saving csv with sentiment analysis
df.to_csv("csv/df_sentiment_analysis.csv")

## Importing the Updated DataFrame

Now let's import the DataFrame again with the sentiment analysis and check if the results make sense

In [24]:
# Importing DataFrame with new Polarity column
df = pd.read_csv('csv/df_sentiment_analysis.csv', index_col=0)

In [25]:
# Checking columns
df.columns

Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
       'days_since_review', 'lat', 'lng', 'Negative_Review_Clean',
       'Positive_Review_Clean', 'Negative_Review_Clean2', 'Polarity_Net',
       'Polarity_Pos'],
      dtype='object')

In [26]:
# Creating function to classify the Sentiment Analysis
df['Sent_Analysis_Neg'] = df['Polarity_Net'].apply(lambda x: 0 if x < 0 else 1 if x > -0.1 and x < 0.1 else 2)
df['Sent_Analysis_Pos'] = df['Polarity_Pos'].apply(lambda x: 0 if x < 0 else 1 if x > -0.1 and x < 0.1 else 2)

In [27]:
# Creating a csv file with the sentiment analysis
sentiment_analysis = df[['Hotel_Name','Negative_Review','Positive_Review','Reviewer_Score','Sent_Analysis_Neg','Sent_Analysis_Pos']]

# Uncomment cell below to export file
# sentiment_analysis.to_csv('sentiment_analysis.csv')

### Findings and Takeaways:

- It was created Subjectivity and Polarity features using sentiment analysis for Negative and Positive Reviews. 
- Polarity ranges between -1 and 1. Where -1 means that the review was very negative and 1 means that the review was very positive.
- Seems like sentiment analysis does a good job identifying positive reviews, but the negative reviews could be improved.

## Target Variable

In this section, I will create a target variable and use it to train my models. I will turn the Reviewer Score classes feature into:

- <b>Bad:</b> Scores below 5
- <b>Regular:</b> Scores between 5 and 7
- <b>Good:</b> Scores above 7

In [28]:
# Checking dataframe
df.head(1)

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Negative_Review_Clean,Positive_Review_Clean,Negative_Review_Clean2,Polarity_Net,Polarity_Pos,Sent_Analysis_Neg,Sent_Analysis_Pos
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,i am so angry that i made this post available...,i am so angry that i made this post available...,i am so angry that i made this post available ...,0.028671,0.283333,1,2


In [29]:
# Create function that turns the Reviewer Score into a classification target with 3 values
df['Score'] = df['Reviewer_Score'].apply(lambda x: 0 if x < 5 else 1 if x >= 5 and x < 7 else 2)

In [30]:
# Checking if function worked
df[['Reviewer_Score', 'Score']].head(5)

,Reviewer_Score,Score
0,2.9,0
1,7.5,2
2,7.1,2
3,3.8,0
4,6.7,1


In [31]:
# Checking if there will be class imbalance
df.Score.value_counts()

2    428887
1     64570
0     22281
Name: Score, dtype: int64

In [32]:
df_Score_2 = df[df.Score == 2].sample(frac=0.05)
df_Score_0 = df[df.Score == 0]
df_Score_1 = df[df.Score == 1].sample(frac=0.3)

In [33]:
df = pd.concat([df_Score_2, df_Score_1, df_Score_0])
df.shape

(63096, 25)

In [34]:
df.Score.value_counts()

0    22281
2    21444
1    19371
Name: Score, dtype: int64

### Findings and Takeaways:

- There is class imbalance in the target variable. Since the dataset if very large, it should not be a problem use downsampling or upsampling.

# Vanilla Model

Before preparing the data for model sampling, I have two problems to solve:

- <b>Class imbalance:</b> There is a big class imbalance in my data set, where there are more positive reviews than negative reviews.
- <b>Curse of Dimensionality:</b> Since my dataset is so large, I'd end up with more than 50,000 features on top of 515K rows. That's too much for my computer to handle with.

Luckily, I might be able to solve both problems at the same time. First, I will balance my target manually, which will help me solve the Curse of Dimensionality partially. Then, I will use Sparse on tokenized dataset.

In [35]:
# Evaluation function

def evaluation(y_true, y_pred):
       
# Print Accuracy, Recall, F1 Score, and Precision metrics.
    print('Evaluation Metrics:')
    print('Accuracy: ' + str(metrics.accuracy_score(y_test, y_pred)))
    print('F1 Score: ' + str(metrics.f1_score(y_test, y_pred, average="macro")))

## Vectorizing Dataset

In [36]:
# Instantiate Stop Words
# add_stop_words = ['mention', 'link', 'rt', 'quot', 'amp', 'sxsw']
stop_words = text.ENGLISH_STOP_WORDS.union()

In [37]:
df.columns

Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
       'days_since_review', 'lat', 'lng', 'Negative_Review_Clean',
       'Positive_Review_Clean', 'Negative_Review_Clean2', 'Polarity_Net',
       'Polarity_Pos', 'Sent_Analysis_Neg', 'Sent_Analysis_Pos', 'Score'],
      dtype='object')

In [53]:
df

(63096, 25)

In [38]:
df.Negative_Review_Clean2

435851                     nothing they have pretty much it
208586    rude bartender chatting away and gave an look ...
401152                             room temperature control
362721                                          no negative
310926                                          no negative
                                ...                        
515661    can t even stop the car to unload in front of ...
515677                         being hotel except breakfast
515700    room vas in different building with no recepti...
515728                                           everything
515735    the was useless it was a hot week in and it on...
Name: Negative_Review_Clean2, Length: 63096, dtype: object

In [39]:
df['Negative_Review_Clean2'] = df['Negative_Review_Clean2'].values.astype('U')

In [40]:
# Instantiate CountVectorizer
cv = CountVectorizer(stop_words=stop_words)

# Fit and transform dataframe without data cleaning
df_cv = cv.fit_transform(df.Negative_Review_Clean2)
df_tk = pd.DataFrame(df_cv.toarray(), columns = cv.get_feature_names())
df_tk.index = df.index

In [41]:
# Using Sparse in the DataFrame
df_sparse = df_tk.astype('Sparse')

In [ ]:
df_sparse.to_csv('sparse_data.csv')

In [42]:
y = df.Score
X = df_sparse

In [43]:
X.to_csv('X_sparse.csv')

(63096, 10930)

## Train Test Split

In [44]:
# Running Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25)

# This cell takes around 4 minutes to run

In [45]:
y_train.shape

(47322,)

## Vanilla Models

### First Model

In [46]:
# Baseline Regression Model
logreg_base = LogisticRegression()
logreg_base.fit(X_train, y_train) 
y_logreg_base = logreg_base.predict(X_test)

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [47]:
# Logistic Regression baseline evaluation
evaluation(y_test, y_logreg_base)

Evaluation Metrics:
Accuracy: 0.5867249904906808
F1 Score: 0.5748068707508759


In [48]:
# Baseline Regression Model
logreg_base = LogisticRegression(max_iter=1000)
logreg_base.fit(X_train, y_train) 
y_logreg_base = logreg_base.predict(X_test)

In [49]:
# Logistic Regression baseline evaluation
evaluation(y_test, y_logreg_base)

Evaluation Metrics:
Accuracy: 0.5874857360213008
F1 Score: 0.5755692278870116


# Pickle DataFrame

In [50]:
# Pickle DataFrame
pd.to_pickle(df, "./dummy.pkl")

# Ideas

- Check if the review is worse if it takes time to be made
- Check the country and nationalities
- Time of the year with more complaints

# Stretch Goals

- Get latitude and longitude for hotels that are missing this information
- People might base their review on an isolated bad experience

In [51]:
''' getting hotels latitude and longetude '''

from geopy.extra.rate_limiter import RateLimiter
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['ADDRESS'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

NameError: name 'locator' is not defined